# Week 7: Large Language Model Systems (Solution)

Complete solutions for Week 7 exercises.

In [ ]:
import os
import json
from typing import Dict, List, Optional, Tuple
from openai import OpenAI
import tiktoken
from tenacity import retry, stop_after_attempt, wait_exponential
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Part 1: LLM Client - SOLUTION

In [ ]:
class LLMClient:
    def __init__(self, model: str = 'gpt-3.5-turbo', api_key: Optional[str] = None):
        self.model = model
        self.client = OpenAI(api_key=api_key or os.getenv('OPENAI_API_KEY'))
        self.tokenizer = tiktoken.encoding_for_model(model)
    
    def count_tokens(self, text: str) -> int:
        return len(self.tokenizer.encode(text))
    
    @retry(stop=stop_after_attempt(3), wait=wait_exponential(min=1, max=10))
    def complete(self, prompt: str, temperature: float = 0.7, max_tokens: int = 500) -> str:
        response = self.client.completions.create(
            model=self.model,
            prompt=prompt,
            temperature=temperature,
            max_tokens=max_tokens
        )
        return response.choices[0].text.strip()
    
    def chat_complete(self, messages: List[Dict], temperature: float = 0.7, max_tokens: int = 500) -> str:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        return response.choices[0].message.content

## Part 2: Prompt Engineering - SOLUTION

In [ ]:
class PromptTemplate:
    def __init__(self, template: str, variables: List[str]):
        self.template = template
        self.variables = variables
    
    def format(self, **kwargs) -> str:
        for var in self.variables:
            if var not in kwargs:
                raise ValueError(f'Missing variable: {var}')
        return self.template.format(**kwargs)

class PromptLibrary:
    @staticmethod
    def zero_shot(task: str, input_text: str) -> str:
        return f'{task}\n\nInput: {input_text}\n\nOutput:'
    
    @staticmethod
    def few_shot(task: str, examples: List[Tuple[str, str]], input_text: str) -> str:
        prompt = f'{task}\n\nExamples:\n'
        for inp, out in examples:
            prompt += f'Input: {inp}\nOutput: {out}\n\n'
        prompt += f'Now your turn:\nInput: {input_text}\nOutput:'
        return prompt
    
    @staticmethod
    def chain_of_thought(problem: str) -> str:
        return f'{problem}\n\nLet\'s think step by step:'

## Part 3: Output Evaluation - SOLUTION

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class OutputEvaluator:
    def check_relevance(self, output: str, query: str) -> float:
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([query, output])
        return float(cosine_similarity(vectors[0:1], vectors[1:2])[0][0])
    
    def check_coherence(self, output: str) -> float:
        sentences = output.split('.')
        if len(sentences) < 2:
            return 1.0
        vectorizer = TfidfVectorizer()
        try:
            vectors = vectorizer.fit_transform(sentences)
            similarities = []
            for i in range(len(sentences)-1):
                sim = cosine_similarity(vectors[i:i+1], vectors[i+1:i+2])[0][0]
                similarities.append(sim)
            return float(np.mean(similarities))
        except:
            return 0.5
    
    def check_factuality(self, output: str, reference: Optional[str] = None) -> float:
        if reference is None:
            return 0.5
        return self.check_relevance(output, reference)

## Part 4: AI Writing Assistant - SOLUTION

In [ ]:
class AIWritingAssistant:
    def __init__(self, model: str = 'gpt-3.5-turbo'):
        self.client = LLMClient(model)
        self.evaluator = OutputEvaluator()
    
    def generate_content(self, topic: str, style: str = 'professional', length: int = 500) -> str:
        prompt = f'Write a {style} {length}-word article about: {topic}'
        return self.client.complete(prompt, max_tokens=length*2)
    
    def improve_text(self, text: str, aspect: str = 'clarity') -> str:
        prompt = f'Improve the {aspect} of this text:\n\n{text}\n\nImproved version:'
        return self.client.complete(prompt)
    
    def analyze_text(self, text: str) -> Dict:
        prompt = f'Analyze this text and provide feedback on clarity, coherence, and style:\n\n{text}'
        analysis = self.client.complete(prompt)
        return {
            'analysis': analysis,
            'coherence_score': self.evaluator.check_coherence(text)
        }
    
    def summarize(self, text: str, max_length: int = 100) -> str:
        prompt = f'Summarize in {max_length} words:\n\n{text}'
        return self.client.complete(prompt, max_tokens=max_length*2)